In [3]:
import pandas as pd
import numpy as np
import sys, os
from IPython.display import display
from geopy.geocoders import Nominatim
from pymongo import MongoClient

In [4]:
place = input ("Enter Address :") 
print(place)

Enter Address :408 brook pine, apex
408 brook pine, apex


In [5]:
#place = "408 Brook Pine Trl, apex"
geolocator = Nominatim(user_agent="Test")
location = geolocator.geocode(place) 
print(location)

408, Echo Park Place, Apex, Chatham County, North Carolina, 27523, United States


In [6]:
print((location.latitude, location.longitude))

(35.7886336, -78.9377002)


In [7]:
  # Declare all variables as strings. Spaces must be replaced with '+', i.e., change 'John Smith' to 'John+Smith'.
  # Define the lat, long of the location and the year
  #lat, lon, year = 33.2164, -97.1292, 2018
  lat, lon =  location.latitude, location.longitude
  # You must request an NSRDB api key from the link above
  api_key = '5qyFRrBVjEZIGuR0WEcihqCEcg4LV8DbErgE6rze'
  # Set the attributes to extract (e.g., dhi, ghi, etc.), separated by commas.
  attributes = 'ghi,dhi,dni,wind_speed,air_temperature,solar_zenith_angle'
  # Choose year of data
  year = '2018'
  # Set leap year to true or false. True will return leap day data if present, false will not.
  leap_year = 'false'
  # Set time interval in minutes, i.e., '30' is half hour intervals. Valid intervals are 30 & 60.
  interval = '30'
  # Specify Coordinated Universal Time (UTC), 'true' will use UTC, 'false' will use the local time zone of the data.
  # NOTE: In order to use the NSRDB data in SAM, you must specify UTC as 'false'. SAM requires the data to be in the
  # local time zone.
  utc = 'false'
  # Your full name, use '+' instead of spaces.
  your_name = 'Narsimha+N'
  # Your reason for using the NSRDB.
  reason_for_use = 'testing'
  # Your affiliation
  your_affiliation = 'ECU'
  # Your email address
  your_email = 'nsnaraya@outlook.com'
  # Please join our mailing list so we can keep you up-to-date on new developments.
  mailing_list = 'false'

  # Declare url string
  url = 'https://developer.nrel.gov/api/solar/nsrdb_psm3_download.csv?wkt=POINT({lon}%20{lat})&names={year}&leap_day={leap}&interval={interval}&utc={utc}&full_name={name}&email={email}&affiliation={affiliation}&mailing_list={mailing_list}&reason={reason}&api_key={api}&attributes={attr}'.format(year=year, lat=lat, lon=lon, leap=leap_year, interval=interval, utc=utc, name=your_name, email=your_email, mailing_list=mailing_list, affiliation=your_affiliation, reason=reason_for_use, api=api_key, attr=attributes)
  # Return just the first 2 lines to get metadata:
  info = pd.read_csv(url, nrows=1)
  # See metadata for specified properties, e.g., timezone and elevation
  timezone, elevation = info['Local Time Zone'], info['Elevation']
  
  df = pd.read_csv(url, nrows=2)
    

In [8]:
# View metadata
info

,Source,Location ID,City,State,Country,Latitude,Longitude,Time Zone,Elevation,Local Time Zone,...,Cloud Type 11,Cloud Type 12,Fill Flag 0,Fill Flag 1,Fill Flag 2,Fill Flag 3,Fill Flag 4,Fill Flag 5,Surface Albedo Units,Version
0,NSRDB,1090003,-,-,-,35.77,-78.94,-5,95,-5,...,Dust,Smoke,NaN,Missing Image,Low Irradiance,Exceeds Clearsky,Missing CLoud Properties,Rayleigh Violation,NaN,3.1.0


In [9]:
# Return all but first 2 lines of csv to get data:
df = pd.read_csv(url, skiprows=2)
    

# Set the time index in the pandas dataframe:
df = df.set_index(pd.date_range('1/1/{yr}'.format(yr=year), freq=interval+'Min', periods=525600/int(interval)))

# take a look
df.shape

(17520, 11)

In [15]:
df.head()

,Year,Month,Day,Hour,Minute,GHI,DHI,DNI,Wind Speed,Temperature,Solar Zenith Angle
2018-01-01 00:00:00,2018,1,1,0,0,0,0,0,1.0,-5.9,166.57
2018-01-01 00:30:00,2018,1,1,0,30,0,0,0,1.1,-6.0,167.00
2018-01-01 01:00:00,2018,1,1,1,0,0,0,0,1.2,-6.2,164.44
2018-01-01 01:30:00,2018,1,1,1,30,0,0,0,1.2,-6.4,160.01
2018-01-01 02:00:00,2018,1,1,2,0,0,0,0,1.2,-6.6,154.69


In [10]:
# Print column names
df.columns.values

array(['Year', 'Month', 'Day', 'Hour', 'Minute', 'GHI', 'DHI', 'DNI',
       'Wind Speed', 'Temperature', 'Solar Zenith Angle'], dtype=object)

In [11]:
df.to_csv("sample.csv")

In [13]:
# Connect to MongoDB
client =  MongoClient("mongodb://localhost:27017/?readPreference=primary&appname=MongoDB%20Compass&ssl=false")

In [14]:
db = client['Solar']
collection = db['Local']

In [15]:
df.reset_index(inplace=True)
data_dict = df.to_dict("records")

In [16]:
# Insert collection
collection.insert_many(data_dict)